In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

from KNN.UserKNNCBFRecommender import UserKNNCBFRecommender


#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [93]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_all = scipy.sparse.load_npz('./Matrix/URM_all_matrix.npz')
ICM_all = scipy.sparse.load_npz('./Matrix/ICM_all_matrix.npz')
URM_train = scipy.sparse.load_npz('./Matrix/URM_train_matrix.npz')
URM_test = scipy.sparse.load_npz('./Matrix/URM_test_matrix.npz')



#Load playlistsIDS
train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

In [95]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of item/user/confidence weights
item_user_data=URM_train.T
print("ITEM USER ={}".format(item_user_data.shape))
model.fit(item_user_data)

# recommend items for a user
user_items = item_user_data.T.tocsr()
print("USER ITEM ={}".format(user_items.shape))


#uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
rec_matrix=np.zeros((50446,10))

for u in all_userID:
    rec = model.recommend(u, user_items)
    for x in range (10):
        rec_matrix[u,x]=rec[x][0]
print(rec_matrix)   

  0%|          | 0/15 [00:00<?, ?it/s]

ITEM USER =(20635, 50446)


100%|██████████| 15.0/15 [00:04<00:00,  3.11it/s]

USER ITEM =(50446, 20635)


In [102]:
import numpy as np
import scipy.sparse as sps



def precision(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    return precision_score



def recall(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]

    return recall_score



def MAP(is_relevant, relevant_items):

    #is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))

    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [103]:
def evaluate_algorithm(URM_test, recMatrix, at=10):

    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0

    num_eval = 0

    URM_test = sps.csr_matrix(URM_test)

    n_users = URM_test.shape[0]


    for user_id in range(n_users):

        if user_id % 10000 == 0:
            print("Evaluated user {} of {}".format(user_id, n_users))

        start_pos = URM_test.indptr[user_id]
        end_pos = URM_test.indptr[user_id+1]

        if end_pos-start_pos>0:

            relevant_items = URM_test.indices[start_pos:end_pos]

            recommended_items = recMatrix[user_id]
            num_eval+=1

            is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

            cumulative_precision += precision(is_relevant, relevant_items)
            cumulative_recall += recall(is_relevant, relevant_items)
            cumulative_MAP += MAP(is_relevant, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval

    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))

    result_dict = {
        "precision": cumulative_precision,
        "recall": cumulative_recall,
        "MAP": cumulative_MAP,
    }

    return result_dict

In [104]:
evaluate_algorithm(URM_test,rec_matrix)

Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0506, Recall = 0.1034, MAP = 0.0449


{'MAP': 0.04492715809434017,
 'precision': 0.05060975609756286,
 'recall': 0.10337629841596505}

In [ ]:
import implicit

factors=[10,50,100,200,500]
reg=[0.01, 0.001,0.1,0.0001]
it=[5,20,40]
item_user_data=URM_train.T
user_items = item_user_data.T.tocsr()





for f in factors:
    for r in reg:
        for i in it:
            print("fator={}, regularization={}, iterations={}".format(f,r,i))
            # initialize a model
            model = implicit.als.AlternatingLeastSquares(factors=f, regularization=r,iterations=i)

            # train the model on a sparse matrix of item/user/confidence weights
            #print("ITEM USER ={}".format(item_user_data.shape))
            model.fit(item_user_data)

            # recommend items for a user
            #print("USER ITEM ={}".format(user_items.shape))


            #uno per volta predico tutti gli utenti e estraggo dalla lista delle 10 predizioni, dalla tupla, la track
            rec_matrix=np.zeros((50446,10))

            for u in all_userID:
                rec = model.recommend(u, user_items)
                for x in range (10):
                    rec_matrix[u,x]=rec[x][0]
            #print(rec_matrix)   
            evaluate_algorithm(URM_test,rec_matrix)

 10%|█         | 0.5/5 [00:00<00:01,  4.26it/s]

fator=10, regularization=0.01, iterations=5


100%|██████████| 5.0/5 [00:01<00:00,  3.61it/s]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446


  0%|          | 0/20 [00:00<?, ?it/s]

...

100%|██████████| 20.0/20 [00:16<00:00,  1.12it/s]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0540, Recall = 0.1181, MAP = 0.0509
fator=200, regularization=0.0001, iterations=40


100%|██████████| 40.0/40 [00:30<00:00,  1.18it/s]


Evaluated user 0 of 50446
Evaluated user 10000 of 50446
Evaluated user 20000 of 50446
Evaluated user 30000 of 50446
Evaluated user 40000 of 50446
Evaluated user 50000 of 50446
Recommender performance is: Precision = 0.0008, Recall = 0.0017, MAP = 0.0003
fator=500, regularization=0.01, iterations=5


100%|██████████| 5.0/5 [00:14<00:00,  3.02s/it]
